In [1]:
import pandas as pd
import numpy as np
import os
import re
import work_files.functions as wf

In [2]:
data = pd.read_csv("../your-code/input/GSAF5.csv", encoding="latin")

display(data.head())
display(data.shape)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


(5992, 24)

In [3]:
wf.check_nulls(data)

(Country                     43
 Area                       402
 Location                   496
 Activity                   527
 Name                       200
 Sex                        567
 Age                       2681
 Injury                      27
 Fatal (Y/N)                 19
 Time                      3213
 Species                   2934
 Investigator or Source      15
 href formula                 1
 href                         3
 Unnamed: 22               5991
 Unnamed: 23               5990
 dtype: int64,
 Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
        'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
        'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
        'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
        'Unnamed: 23'],
       dtype='object'),
 (5992, 24))

In [4]:
data = pd.read_csv("../your-code/input/GSAF5.csv", encoding="latin")

data = data.drop(['Time','pdf','href formula', 'href','Unnamed: 22', 'Unnamed: 23'],axis=1)
data = data.rename(columns = {"Species ":"Species","Fatal (Y/N)":"Fatal"})

display(data.head())
display(data.shape)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Species,Investigator or Source,Case Number.1,Case Number.2,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c,2016.09.18.c,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b,2016.09.18.b,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a,2016.09.18.a,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,"The Age, 9/18/2016",2016.09.17,2016.09.17,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,2 m shark,"The Age, 9/16/2016",2016.09.16,2016.09.15,5989


(5992, 18)

In [5]:
data = data[data["Year"] >= 1950]

In [6]:
wf.check_nulls(data)

(Country                     13
 Area                       205
 Location                   260
 Activity                   313
 Name                       104
 Sex                        394
 Age                       1396
 Injury                      15
 Fatal                        8
 Species                   1653
 Investigator or Source      12
 dtype: int64,
 Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
        'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal', 'Species',
        'Investigator or Source', 'Case Number.1', 'Case Number.2',
        'original order'],
       dtype='object'),
 (4271, 18))

In [7]:
data["Type"].value_counts()

Unprovoked      3193
Provoked         417
Invalid          353
Boat             194
Sea Disaster      95
Boating           19
Name: Type, dtype: int64

In [8]:
data = data[data["Type"] != "Invalid"]

In [9]:
wf.check_nulls(data)

(Country                     10
 Area                       184
 Location                   231
 Activity                   246
 Name                        76
 Sex                        348
 Age                       1236
 Injury                      11
 Fatal                        4
 Species                   1503
 Investigator or Source      10
 dtype: int64,
 Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
        'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal', 'Species',
        'Investigator or Source', 'Case Number.1', 'Case Number.2',
        'original order'],
       dtype='object'),
 (3918, 18))

In [10]:
data = data.drop(["Case Number.1","Case Number.2","original order","Investigator or Source"],axis=1)

In [11]:
wf.check_nulls(data)

(Country       10
 Area         184
 Location     231
 Activity     246
 Name          76
 Sex          348
 Age         1236
 Injury        11
 Fatal          4
 Species     1503
 dtype: int64,
 Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
        'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal', 'Species'],
       dtype='object'),
 (3918, 14))

In [12]:
display(data[data["Fatal"].isnull()])

data[data["Fatal"].isnull()].index

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Species
54,2016.06.02.b,02-Jun-16,2016,Unprovoked,AUSTRALIA,New South Wales,Kingscliff,Spearfishing,Waade Madigan and Dr Seyong Kim,M,NaN,"No injury, but sharks repeatedly hit their fin...",NaN,Bronze whaler sharks x 3
3280,1967.09.13,13-Sep-67,1967,Provoked,ITALY,Brindisi Province,Brindisi,Scuba diving,Romeo Guarini,M,21,"Diver shot the shark, then it injured his arm ...",NaN,2 m shark
3901,1958.12.28,28-Dec-58,1958,Boat,TURKEY,Ahirkapi coast,Constantinople,Fishing,Fishing boat. Occupants: Yunus Potur & Ali Durmaz,NaN,NaN,Boat damaged,NaN,White shark
4107,1954.07.27,27-Jul-54,1954,Boating,ITALY,Venice Province,Off Chioggia,Fishing trawler Flavio Gioia,10 crew,M,NaN,No injury to occupants. Shark tore nets & traw...,NaN,5m shark


Int64Index([54, 3280, 3901, 4107], dtype='int64')

In [13]:
data.loc[54,"Fatal"] = "N"
data.loc[3280,"Fatal"] = "N"
data.loc[3901,"Fatal"] = "N"
data.loc[4107,"Fatal"] = "N"

In [14]:
wf.check_nulls(data)

(Country       10
 Area         184
 Location     231
 Activity     246
 Name          76
 Sex          348
 Age         1236
 Injury        11
 Species     1503
 dtype: int64,
 Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
        'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal', 'Species'],
       dtype='object'),
 (3918, 14))

In [15]:
data["Fatal"].value_counts()

N          3253
Y           615
UNKNOWN      43
 N            7
Name: Fatal, dtype: int64

In [16]:
data = data[data["Fatal"] != "UNKNOWN"]
data["Fatal"] = data["Fatal"].apply(wf.espacio)

In [17]:
data["Fatal"].value_counts()

N    3260
Y     615
Name: Fatal, dtype: int64

In [18]:
data["Country"].value_counts()

USA                     1621
AUSTRALIA                669
SOUTH AFRICA             406
PAPUA NEW GUINEA         108
BRAZIL                    86
                        ... 
UNITED ARAB EMIRATES       1
NORTH SEA                  1
Sierra Leone               1
BRITISH ISLES              1
NORWAY                     1
Name: Country, Length: 139, dtype: int64

In [19]:
keys = data["Country"].value_counts().index
values = data["Country"].value_counts().values

countries = {keys[i]: values[i] for i in range(len(keys))}
drop_countries = []

for i in countries:
    if countries[i] < 10:
        drop_countries.append(i)
    else:
        pass
     

for c in drop_countries:
    data = data[data["Country"] != c]

In [20]:
display(data.head())

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Species
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,2 m shark


In [21]:
wf.check_nulls(data)

(Country       10
 Area         101
 Location     153
 Activity     197
 Name          46
 Sex          280
 Age         1037
 Species     1312
 dtype: int64,
 Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
        'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal', 'Species'],
       dtype='object'),
 (3519, 14))

In [22]:
data = data.dropna(subset=["Country"])
data = data.drop(["Area","Location"],axis=1)

In [23]:
wf.check_nulls(data)

(Activity     194
 Name          45
 Sex          279
 Age         1030
 Species     1306
 dtype: int64,
 Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Activity', 'Name',
        'Sex ', 'Age', 'Injury', 'Fatal', 'Species'],
       dtype='object'),
 (3509, 12))

In [31]:
data["Activity"].value_counts()

0                                                 Surfing
1                                                 Surfing
2                                                 Surfing
3                                                 Surfing
4                                                 Surfing
                              ...                        
4263                                             Standing
4265                                             Bathing 
4268    Spearfishing, but walking carrying fish on end...
4269              Helmet diving, collecting trochus shell
4270                                                  NaN
Name: Activity, Length: 3509, dtype: object

In [43]:
def surf(text):
    if re.search('(urfing)\w+', text):
        return 'Surfing'
    return text

In [44]:
surf(data["Activity"])

TypeError: expected string or bytes-like object